In [2]:
import wrds
import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.stats import gmean
from matplotlib import pyplot as plt
import riskfolio as rp

conn = wrds.Connection(wrds_username='jackw')

Loading library list...
Done


In [3]:
input_portfolio = ["NVDA", "AAPL", "MSFT", "LUV", "AMZN", "MET", "AMD", "SPG", "EQIX", "APD", "CRH", "CVX", "REGN", "VRTX", "SBAC", "ORLY", "DAL", "ETN", "INTC", "SO", "DUK", "NOC", "PCAR", "FITB", "EBAY"]
portfolio = sorted(input_portfolio)

def training_data_retrieval(portfolio):
    portfolio_data = conn.raw_sql("""select ticker, mthcaldt, mthret 
                            from crsp_a_stock.wrds_msfv2_query 
                            where mthcaldt>='01/01/2019'
                            and mthcaldt<= '12/31/2020'""", 
                         date_cols=['mthcaldt'])
    portfolio_data = portfolio_data[portfolio_data.ticker.isin(portfolio) == True]
    portfolio_df = portfolio_data.pivot_table(index='mthcaldt', columns='ticker', values='mthret', aggfunc='mean')
    return portfolio_df

training_returns = training_data_retrieval(portfolio)

def testing_data_retrieval(portfolio):
    portfolio_data = conn.raw_sql("""select ticker, mthcaldt, mthret 
                            from crsp_a_stock.wrds_msfv2_query 
                            where mthcaldt>='01/01/2021'
                            and mthcaldt<= '12/31/2021'""", 
                         date_cols=['mthcaldt'])
    portfolio_data = portfolio_data[portfolio_data.ticker.isin(portfolio) == True]
    portfolio_df = portfolio_data.pivot_table(index='mthcaldt', columns='ticker', values='mthret', aggfunc='mean')
    return portfolio_df

testing_returns = testing_data_retrieval(portfolio)

In [4]:
def plot_dendrogram(portfolio):
    '''
    Plots dendrogram depicting clusters and the distances, representing the similarities
    '''
    ax = rp.plot_dendrogram(returns=training_returns, codependence='distance', linkage='ward')
    return ax

def hrp_allocation(portfolio):
    '''
    Uses inverse variation to allocate weights to each stock based on its cluster's variance
    '''
    port = rp.HCPortfolio(returns=training_returns)
    hrp_weights = port.optimization(model="HRP", codependence="distance", rm="MV", linkage="ward")
    return hrp_weights

def equal_allocation(portfolio):
    '''
    Creates a list of equal allocations based on the number of stocks in the portfolio
    '''
    equal_allocations = []
    for stock in range(len(portfolio)):
        equal_allocations.append( 1 / len(portfolio))
    return equal_allocations

def minimum_variance_allocation(portfolio):
    port = rp.Portfolio(returns=training_returns)
    port.assets_stats(method_mu="hist", method_cov="hist")
    minimum_variance_weights = port.optimization(model="Classic", rm="MV", obj="MinRisk")
    return minimum_variance_weights

def mean_variance_optimal_allocation(portfolio):
    port = rp.Portfolio(returns=training_returns)
    port.assets_stats(method_mu="hist", method_cov="hist")
    mean_variance_optimal_weights = port.optimization(model="Classic", rm="MV", obj="Sharpe")
    return mean_variance_optimal_weights

def gmean_return(portfolio_df):
    '''
    Calculates geometric mean return for each stock in the portfolio
    '''
    gmean_return = 0
    portfolio_df = portfolio_df + 1
    gmean_return = gmean(portfolio_df)
    gmean_return = gmean_return - 1
    return gmean_return

In [5]:
# HRP Testing

def hrp_training_return(portfolio):
    '''
    Uses geometric mean returns of each stock to determine overall return of portfolio based on allocations
    '''
    gmean_returns = gmean_return(training_returns)
    hrp_return = np.dot(gmean_returns, hrp_allocation(portfolio))
    return hrp_return

def hrp_training_risk(portfolio):
    '''
    Uses covariances and allocations to determine the risk of the HRP portfolio
    '''
    cov_matrix = training_returns.cov()
    risk = np.sqrt(np.dot(hrp_allocation(portfolio).T, np.dot(cov_matrix,hrp_allocation(portfolio))))
    stddev_lol = risk.tolist()
    stddev_l = [i[0] for i in stddev_lol]
    stddev = float(stddev_l[0])
    return stddev

def hrp_testing_return(portfolio):
    '''
    Uses geometric mean returns of each stock to determine overall return of portfolio based on allocations
    '''
    gmean_returns = gmean_return(testing_returns)
    hrp_return = np.dot(gmean_returns, hrp_allocation(portfolio))
    return hrp_return

def hrp_testing_risk(portfolio):
    '''
    Uses covariances and allocations to determine the risk of the HRP portfolio
    '''
    cov_matrix = testing_returns.cov()
    risk = np.sqrt(np.dot(hrp_allocation(portfolio).T, np.dot(cov_matrix,hrp_allocation(portfolio))))
    stddev_lol = risk.tolist()
    stddev_l = [i[0] for i in stddev_lol]
    stddev = float(stddev_l[0])
    return stddev

def hrp_cumulative_return_testing(portfolio):
    '''
    Calculates HRP portfolio's cumulative return for testing period
    '''
    cumulative_return = 1
    portfolio_df = testing_returns
    for date in range(len(portfolio_df.index)):
        date_returns = portfolio_df.iloc[date].values.tolist()
        cumulative_return = cumulative_return * (1 + np.dot(hrp_allocation(portfolio).T, date_returns))
    cumulative_return = (cumulative_return - 1)
    return cumulative_return

def hrp_return_testing_list(portfolio):
    portfolio_df = testing_returns
    monthly_returns = []
    cumulative_return = 1
    for date in range(len(portfolio_df.index)):
        date_returns = portfolio_df.iloc[date].values.tolist()
        cumulative_return = cumulative_return * (1 + np.dot(hrp_allocation(portfolio).T, date_returns))
        monthly_returns.append(cumulative_return - 1)
    return monthly_returns

def hrp_cumulative_return_training(portfolio):
    '''
    Calculates HRP portfolio's cumulative return for training period
    '''
    cumulative_return = 1
    portfolio_df = training_returns
    for date in range(len(portfolio_df.index)):
        date_returns = portfolio_df.iloc[date].values.tolist()
        cumulative_return = cumulative_return * (1 + np.dot(hrp_allocation(portfolio).T, date_returns))
    cumulative_return = (cumulative_return - 1)
    return cumulative_return

def hrp_return_training_list(portfolio):
    portfolio_df = training_returns
    monthly_returns = []
    cumulative_return = 1
    for date in range(len(portfolio_df.index)):
        date_returns = portfolio_df.iloc[date].values.tolist()
        cumulative_return = cumulative_return * (1 + np.dot(hrp_allocation(portfolio).T, date_returns))
        monthly_returns.append(cumulative_return - 1)
    return monthly_returns

# Minimum-Variance Testing

def mv_training_return(portfolio):
    '''
    Uses geometric mean returns of each stock to determine overall return of portfolio based on allocations
    '''
    gmean_returns = gmean_return(training_returns)
    mv_return = np.dot(gmean_returns, minimum_variance_allocation(portfolio))
    return mv_return

def mv_training_risk(portfolio):
    '''
    Uses covariances and allocations to determine the risk of the Minimum-Variance portfolio
    '''
    cov_matrix = training_returns.cov()
    risk = np.sqrt(np.dot(minimum_variance_allocation(portfolio).T, np.dot(cov_matrix,minimum_variance_allocation(portfolio))))
    stddev_lol = risk.tolist()
    stddev_l = [i[0] for i in stddev_lol]
    stddev = float(stddev_l[0])
    return stddev

def mv_testing_return(portfolio):
    '''
    Uses geometric mean returns of each stock to determine overall return of portfolio based on allocations
    '''
    gmean_returns = gmean_return(testing_returns)
    mv_return = np.dot(gmean_returns, minimum_variance_allocation(portfolio))
    return mv_return

def mv_testing_risk(portfolio):
    '''
    Uses covariances and allocations to determine the risk of the Minimum-Variance portfolio
    '''
    cov_matrix = testing_returns.cov()
    risk = np.sqrt(np.dot(minimum_variance_allocation(portfolio).T, np.dot(cov_matrix,minimum_variance_allocation(portfolio))))
    stddev_lol = risk.tolist()
    stddev_l = [i[0] for i in stddev_lol]
    stddev = float(stddev_l[0])
    return stddev

def mv_cumulative_return_testing(portfolio):
    '''
    Calculates Minimimum Variance portfolio's cumulative return for testing period
    '''
    cumulative_return = 1
    portfolio_df = testing_returns
    for date in range(len(portfolio_df.index)):
        date_returns = portfolio_df.iloc[date].values.tolist()
        cumulative_return = cumulative_return * (1 + np.dot(minimum_variance_allocation(portfolio).T, date_returns))
    cumulative_return = (cumulative_return - 1)
    return cumulative_return

def mv_return_testing_list(portfolio):
    portfolio_df = testing_returns
    monthly_returns = []
    cumulative_return = 1
    for date in range(len(portfolio_df.index)):
        date_returns = portfolio_df.iloc[date].values.tolist()
        cumulative_return = cumulative_return * (1 + np.dot(minimum_variance_allocation(portfolio).T, date_returns))
        monthly_returns.append(cumulative_return - 1)
    return monthly_returns

def mv_cumulative_return_training(portfolio):
    '''
    Calculates Minimum Variance portfolio's cumulative return for training period
    '''
    cumulative_return = 1
    portfolio_df = training_returns
    for date in range(len(portfolio_df.index)):
        date_returns = portfolio_df.iloc[date].values.tolist()
        cumulative_return = cumulative_return * (1 + np.dot(minimum_variance_allocation(portfolio).T, date_returns))
    cumulative_return = (cumulative_return - 1)
    return cumulative_return

def mv_return_training_list(portfolio):
    portfolio_df = training_returns
    monthly_returns = []
    cumulative_return = 1
    for date in range(len(portfolio_df.index)):
        date_returns = portfolio_df.iloc[date].values.tolist()
        cumulative_return = cumulative_return * (1 + np.dot(minimum_variance_allocation(portfolio).T, date_returns))
        monthly_returns.append(cumulative_return - 1)
    return monthly_returns



# Mean-Variance Optimal Testing

def meanv_training_return(portfolio):
    '''
    Uses geometric mean returns of each stock to determine overall return of portfolio based on allocations
    '''
    gmean_returns = gmean_return(training_returns)
    meanv_return = np.dot(gmean_returns, mean_variance_optimal_allocation(portfolio))
    return meanv_return

def meanv_training_risk(portfolio):
    '''
    Uses covariances and allocations to determine the risk of the Minimum-Variance portfolio
    '''
    cov_matrix = training_returns.cov()
    risk = np.sqrt(np.dot(mean_variance_optimal_allocation(portfolio).T, np.dot(cov_matrix,mean_variance_optimal_allocation(portfolio))))
    stddev_lol = risk.tolist()
    stddev_l = [i[0] for i in stddev_lol]
    stddev = float(stddev_l[0])
    return stddev

def meanv_testing_return(portfolio):
    '''
    Uses geometric mean returns of each stock to determine overall return of portfolio based on allocations
    '''
    gmean_returns = gmean_return(testing_returns)
    meanv_return = np.dot(gmean_returns, mean_variance_optimal_allocation(portfolio))
    return meanv_return

def meanv_testing_risk(portfolio):
    '''
    Uses covariances and allocations to determine the risk of the Minimum-Variance portfolio
    '''
    cov_matrix = testing_returns.cov()
    risk = np.sqrt(np.dot(mean_variance_optimal_allocation(portfolio).T, np.dot(cov_matrix,mean_variance_optimal_allocation(portfolio))))
    stddev_lol = risk.tolist()
    stddev_l = [i[0] for i in stddev_lol]
    stddev = float(stddev_l[0])
    return stddev

def meanv_cumulative_return_testing(portfolio):
    '''
    Calculates Minimimum Variance portfolio's cumulative return for testing period
    '''
    cumulative_return = 1
    portfolio_df = testing_returns
    for date in range(len(portfolio_df.index)):
        date_returns = portfolio_df.iloc[date].values.tolist()
        cumulative_return = cumulative_return * (1 + np.dot(mean_variance_optimal_allocation(portfolio).T, date_returns))
    cumulative_return = (cumulative_return - 1)
    return cumulative_return

def meanv_return_testing_list(portfolio):
    portfolio_df = testing_returns
    monthly_returns = []
    cumulative_return = 1
    for date in range(len(portfolio_df.index)):
        date_returns = portfolio_df.iloc[date].values.tolist()
        cumulative_return = cumulative_return * (1 + np.dot(mean_variance_optimal_allocation(portfolio).T, date_returns))
        monthly_returns.append(cumulative_return - 1)
    return monthly_returns

def meanv_cumulative_return_training(portfolio):
    '''
    Calculates Minimum Variance portfolio's cumulative return for training period
    '''
    cumulative_return = 1
    portfolio_df = training_returns
    for date in range(len(portfolio_df.index)):
        date_returns = portfolio_df.iloc[date].values.tolist()
        cumulative_return = cumulative_return * (1 + np.dot(mean_variance_optimal_allocation(portfolio).T, date_returns))
    cumulative_return = (cumulative_return - 1)
    return cumulative_return

def meanv_return_training_list(portfolio):
    portfolio_df = training_returns
    monthly_returns = []
    cumulative_return = 1
    for date in range(len(portfolio_df.index)):
        date_returns = portfolio_df.iloc[date].values.tolist()
        cumulative_return = cumulative_return * (1 + np.dot(mean_variance_optimal_allocation(portfolio).T, date_returns))
        monthly_returns.append(cumulative_return - 1)
    return monthly_returns




# Equal-Weight Testing

def equal_training_return(portfolio):
    '''
    Uses geometric mean returns of each stock to determine overall return of portfolio based on equal allocations
    '''
    gmean_returns = gmean_return(training_returns)
    equal_return = np.dot(gmean_returns, equal_allocation(portfolio))
    return equal_return

def equal_training_risk(portfolio):
    '''
    Uses covariances and allocations to determine the risk of the equal-weight portfolio
    '''
    cov_matrix = training_returns.cov()
    risk = np.sqrt(np.dot(equal_allocation(portfolio), np.dot(cov_matrix,equal_allocation(portfolio))))
    stddev = risk.tolist()
    return stddev

def equal_testing_return(portfolio):
    '''
    Uses geometric mean returns of each stock to determine overall return of portfolio based on equal allocations
    '''
    gmean_returns = gmean_return(testing_returns)
    equal_return = np.dot(gmean_returns, equal_allocation(portfolio))
    return equal_return

def equal_testing_risk(portfolio):
    '''
    Uses covariances and allocations to determine the risk of the equal-weight portfolio
    '''
    cov_matrix = testing_returns.cov()
    risk = np.sqrt(np.dot(equal_allocation(portfolio), np.dot(cov_matrix,equal_allocation(portfolio))))
    stddev = risk.tolist()
    return stddev

def equal_cumulative_return_testing(portfolio):
    '''
    Calculates equal-weight portfolio's cumulative return for testing period
    '''
    cumulative_return = 1
    portfolio_df = testing_returns
    for date in range(len(portfolio_df.index)):
        date_returns = portfolio_df.iloc[date].values.tolist()
        cumulative_return = cumulative_return * (1 + np.dot(equal_allocation(portfolio), date_returns))
    cumulative_return = (cumulative_return - 1)
    return cumulative_return

def equal_return_testing_list(portfolio):
    portfolio_df = testing_returns
    monthly_returns = []
    cumulative_return = 1
    for date in range(len(portfolio_df.index)):
        date_returns = portfolio_df.iloc[date].values.tolist()
        cumulative_return = cumulative_return * (1 + np.dot(equal_allocation(portfolio), date_returns))
        monthly_returns.append(cumulative_return - 1)
    return monthly_returns

def equal_cumulative_return_training(portfolio):
    '''
    Calculates HRP portfolio's cumulative return for training period
    '''
    cumulative_return = 1
    portfolio_df = training_returns
    for date in range(len(portfolio_df.index)):
        date_returns = portfolio_df.iloc[date].values.tolist()
        cumulative_return = cumulative_return * (1 + np.dot(equal_allocation(portfolio), date_returns))
    cumulative_return = (cumulative_return - 1)
    return cumulative_return

def equal_return_training_list(portfolio):
    portfolio_df = training_returns
    monthly_returns = []
    cumulative_return = 1
    for date in range(len(portfolio_df.index)):
        date_returns = portfolio_df.iloc[date].values.tolist()
        cumulative_return = cumulative_return * (1 + np.dot(equal_allocation(portfolio), date_returns))
        monthly_returns.append(cumulative_return - 1)
    return monthly_returns

def results_df(portfolio):
    hrp_train_return = str(hrp_training_return(portfolio) * 100)[1:6] + "%"
    hrp_train_risk = str(hrp_training_risk(portfolio) * 100)[:5] + "%"
    hrp_cum_return_train = str(hrp_cumulative_return_training(portfolio) * 100)[1:7] + "%"
    hrp_test_return = str(hrp_testing_return(portfolio) * 100)[1:6] + "%"
    hrp_test_risk = str(hrp_testing_risk(portfolio) * 100)[:5] + "%"
    hrp_cum_return_test = str(hrp_cumulative_return_testing(portfolio) * 100)[1:7] + "%"
    mv_train_return = str(mv_training_return(portfolio) * 100)[1:6] + "%"
    mv_train_risk = str(mv_training_risk(portfolio) * 100)[:5] + "%"
    mv_cum_return_train = str(mv_cumulative_return_training(portfolio) * 100)[1:7] + "%"
    mv_test_return = str(mv_testing_return(portfolio) * 100)[1:6] + "%"
    mv_test_risk = str(mv_testing_risk(portfolio) * 100)[:5] + "%"
    mv_cum_return_test = str(mv_cumulative_return_testing(portfolio) * 100)[1:7] + "%"
    meanv_train_return = str(meanv_training_return(portfolio) * 100)[1:6] + "%"
    meanv_train_risk = str(meanv_training_risk(portfolio) * 100)[:5] + "%"
    meanv_cum_return_train = str(meanv_cumulative_return_training(portfolio) * 100)[1:7] + "%"
    meanv_test_return = str(meanv_testing_return(portfolio) * 100)[1:6] + "%"
    meanv_test_risk = str(meanv_testing_risk(portfolio) * 100)[:5] + "%"
    meanv_cum_return_test = str(meanv_cumulative_return_testing(portfolio) * 100)[1:7] + "%"
    equal_train_return = str(equal_training_return(portfolio) * 100)[:5] + "%"
    equal_train_risk = str(equal_training_risk(portfolio) * 100)[:5] + "%"
    equal_cum_return_train = str(equal_cumulative_return_training(portfolio) * 100)[:6] + "%"
    equal_test_return = str(equal_testing_return(portfolio) * 100)[:5] + "%"
    equal_test_risk = str(equal_testing_risk(portfolio) * 100)[:5] + "%"
    equal_cum_return_test = str(equal_cumulative_return_testing(portfolio) * 100)[:6] + "%"
    hrp_train_risk_adjusted_return = str(hrp_training_return(portfolio) / hrp_training_risk(portfolio))[1:7]
    hrp_test_risk_adjusted_return = str(hrp_testing_return(portfolio) / hrp_testing_risk(portfolio))[1:7]
    mv_train_risk_adjusted_return = str(mv_training_return(portfolio) / mv_training_risk(portfolio))[1:7]
    mv_test_risk_adjusted_return = str(mv_testing_return(portfolio) / mv_testing_risk(portfolio))[1:7]
    meanv_train_risk_adjusted_return = str(meanv_training_return(portfolio) / meanv_training_risk(portfolio))[1:7]
    meanv_test_risk_adjusted_return = str(meanv_testing_return(portfolio) / meanv_testing_risk(portfolio))[1:7]
    equal_train_risk_adjusted_return = str(equal_training_return(portfolio) / equal_training_risk(portfolio))[:6]
    equal_test_risk_adjusted_return = str(equal_testing_return(portfolio) / equal_testing_risk(portfolio))[:6]
    result_df = pd.DataFrame([[hrp_train_return, hrp_test_return, equal_train_return, equal_test_return, mv_train_return, mv_test_return, meanv_train_return, meanv_test_return],
                               [hrp_train_risk, hrp_test_risk, equal_train_risk, equal_test_risk, mv_train_risk, mv_test_risk, meanv_train_risk, meanv_test_risk],
                               [hrp_train_risk_adjusted_return, hrp_test_risk_adjusted_return, equal_train_risk_adjusted_return, equal_test_risk_adjusted_return, mv_train_risk_adjusted_return, mv_test_risk_adjusted_return, meanv_train_risk_adjusted_return, meanv_test_risk_adjusted_return],
                               [hrp_cum_return_train, hrp_cum_return_test, equal_cum_return_train, equal_cum_return_test, mv_cum_return_train, mv_cum_return_test, meanv_cum_return_train, meanv_cum_return_test]],
                  index=pd.Index(["Mean Return", "Risk", "Risk-Adjusted Return", "Cumulative Return"], name="Performance Measure:"),
                  columns=pd.MultiIndex.from_product([["HRP Portfolio", "Equal-Weight Portfolio", "Minimum Variance Portfolio", "Mean-Variance Optimal Portfolio"],["Training", "Testing"]], names=['Model:', 'Data Sample:']))
    return result_df